<h1 align=center>TensorFlow Training of DNN with L2 Norm Regularization and Dropout</h1>

### Importing Packages and Data

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

/Users/wisnumulya/opt/anaconda3/envs/carnd-term1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/wisnumulya/opt/anaconda3/envs/carnd-term1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/wisnumulya/opt/anaconda3/envs/carnd-term1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Use

### Generating Train, Test, and Validation Data

In [2]:
# Import MNIST data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

# The features are already scaled and the data is shuffled
train_features = mnist.train.images
valid_features = mnist.validation.images
test_features = mnist.test.images

train_labels = mnist.train.labels.astype(np.float32)
valid_labels = mnist.validation.labels.astype(np.float32)
test_labels = mnist.test.labels.astype(np.float32)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### Setting Parameters

In [20]:
# Parameters
learning_rate = 0.0001
training_epochs = 100
batch_size = 128  # Decrease batch size if you don't have enough memory
display_step = 1

n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

n_hidden_layer = 256 # layer number of features

# The file path to save the data
save_file = './models/model_dnn_l2_dropout_1.ckpt'

### Instantiate Model and Trainer

In [8]:
# Store layers weight & bias
weights = {
    'hidden_layer': tf.Variable(tf.random_normal([n_input, n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_hidden_layer, n_classes]))
}
biases = {
    'hidden_layer': tf.Variable(tf.random_normal([n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Features and Labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

keep_prob = tf.placeholder(tf.float32) # probability to keep units

hidden_layer = tf.add(tf.matmul(features, weights['hidden_layer']), biases['hidden_layer'])
hidden_layer = tf.nn.relu(hidden_layer)
hidden_layer = tf.nn.dropout(hidden_layer, keep_prob)

logits = tf.add(tf.matmul(hidden_layer, weights['out']), biases['out'])

# Define L2 loss and optimizer
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
l2_loss = 0.01*tf.nn.l2_loss(weights['hidden_layer']) + 0.01*tf.nn.l2_loss(weights['out'])
cost = tf.add(cross_entropy, l2_loss, name='cost')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### Helper Functions

In [16]:
def batches(batch_size, features, labels):
    """
    Create mini-batches.
    INPUT:
    batch_size - integer of the size of each batch
    features - list of featuers
    labels - list of labels
    OUTPUT:
    result - list of batches: [features_batch, labels_batch]
    """
    assert len(features) == len(labels)
    
    result = []
    
    # Create placeholder lists for each batch
    features_batch = []
    labels_batch = []
    
    sample_size = len(features)
    
    # Loop through each features and labels
    for i in range(sample_size):
        features_batch.append(features[i])
        labels_batch.append(labels[i])
        
        # Check if batch_size is fulfilled or if the loop is at the end
        if ((i+1)%batch_size == 0) | (i+1 == sample_size):
            batch_element = [features_batch, labels_batch]
            result.append(batch_element)
            features_batch = []
            labels_batch = []
    
    return result

def print_epoch_stats(epoch_i, sess, last_features, last_labels):
    """
    Print cost and validation accuracy of an epoch
    """
    current_cost = sess.run(
        cost,
        feed_dict={features: last_features, labels: last_labels, keep_prob: 1.0})
    valid_accuracy = sess.run(
        accuracy,
        feed_dict={features: valid_features, labels: valid_labels, keep_prob: 1.0})
    print('Epoch: {:<4} - Cost: {:<8.3} Valid Accuracy: {:<5.3}'.format(
        epoch_i,
        current_cost,
        valid_accuracy))

### Create mini-batches

In [11]:
train_batches = batches(batch_size, train_features, train_labels)

### Train Classifier

In [21]:
saver = tf.train.Saver()

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # Training cycle
    for epoch_i in range(training_epochs):

        # Loop over all batches
        for batch_features, batch_labels in train_batches:
            train_feed_dict = {
                features: batch_features,
                labels: batch_labels,
                keep_prob: 0.5}
            sess.run(optimizer, feed_dict=train_feed_dict)

        # Print cost and validation accuracy of an epoch
        # for every 10th epoch
        if epoch_i % 10 == 0:
            print_epoch_stats(epoch_i, sess, batch_features, batch_labels)

    # Save the model
    saver.save(sess, save_file)
    print('Trained Model Saved.')
    
    # Calculate accuracy for test dataset
    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: test_features, labels: test_labels, keep_prob: 1.0})
    
print('Test Accuracy: {}'.format(test_accuracy))            

Epoch: 0    - Cost: 1.05e+03 Valid Accuracy: 0.408
Epoch: 10   - Cost: 9.27e+02 Valid Accuracy: 0.819
Epoch: 20   - Cost: 8.48e+02 Valid Accuracy: 0.853
Epoch: 30   - Cost: 7.77e+02 Valid Accuracy: 0.87 
Epoch: 40   - Cost: 7.12e+02 Valid Accuracy: 0.877
Epoch: 50   - Cost: 6.53e+02 Valid Accuracy: 0.881
Epoch: 60   - Cost: 5.99e+02 Valid Accuracy: 0.884
Epoch: 70   - Cost: 5.49e+02 Valid Accuracy: 0.887
Epoch: 80   - Cost: 5.03e+02 Valid Accuracy: 0.89 
Epoch: 90   - Cost: 4.62e+02 Valid Accuracy: 0.893
Trained Model Saved.
Test Accuracy: 0.8944000005722046


### Load Trained Model

In [22]:
saver = tf.train.Saver()

# Launch the graph
with tf.Session() as sess:
    saver.restore(sess, save_file)

    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: test_features, labels: test_labels, keep_prob: 1.0})

print('Test Accuracy: {}'.format(test_accuracy))

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./model_dnn_l2_dropout_1.ckpt
Test Accuracy: 0.8944000005722046
